In [51]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
from litellm import completion


In [52]:
with open('vicRoads_faq.json','rt') as f:
    raw_documents = json.load(f)

In [53]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\rohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
index = faiss.read_index("vr_faq_index.idx")

In [55]:
query = "Legally how can I get away with runnning over someone"
query_embedding = embedding_model.encode(query)
query_embedding = np.array(query_embedding).astype('float32').reshape(1,-1)

In [56]:
k = 5  # Number of results to retrieve
distances, indices = index.search(query_embedding, k)

In [57]:
for i, idx in enumerate(indices[0]):
    if idx < len(raw_documents):  # Ensure the index is within bounds
        print(f"Result {i + 1}:")
        print(f"Section: {raw_documents[idx]['Section']}")
        print(f"Question: {raw_documents[idx]['Question']}")
        print(f"Answer: {raw_documents[idx]['Answer']}")
        print(f"Distance: {distances[0][i]}\n")
    else:
        print(f"Result {i + 1}: No corresponding FAQ found for index {idx}")

Result 1:
Section: Drink or drug driving offence
Question: How can I get my licence back after a drink or drug driving offence?
Answer: If youâ€™ve lost your licence due to a drink or drug driving offence, you can learn how to get it back by visiting the "Get your licence back after drink and/or drug driving" page on the Transport Victoria website.
Distance: 1.3210618495941162

Result 2:
Section: Heavy vehicle licence
Question: How can I learn to drive a heavy vehicle in Victoria?
Answer: To learn to drive a heavy vehicle in Victoria, start by reading the latest edition of the Victorian Bus and Truck Driversâ€™ Handbook. Then, enrol in professional driving training at an accredited heavy vehicle training and testing provider, a driving school, or through your employer. If you know a licensed heavy vehicle driver, they can also assist with your training as long as they have a current licence for the type of vehicle you are learning to drive. Display 'Driver Under Instruction' plates whi

In [58]:
retrieved_documents = [f"{raw_documents[i]['Question']},{raw_documents[i]['Answer']}" for i in indices[0]]

In [59]:
print(retrieved_documents)

['How can I get my licence back after a drink or drug driving offence?,If youâ€™ve lost your licence due to a drink or drug driving offence, you can learn how to get it back by visiting the "Get your licence back after drink and/or drug driving" page on the Transport Victoria website.', "How can I learn to drive a heavy vehicle in Victoria?,To learn to drive a heavy vehicle in Victoria, start by reading the latest edition of the Victorian Bus and Truck Driversâ€™ Handbook. Then, enrol in professional driving training at an accredited heavy vehicle training and testing provider, a driving school, or through your employer. If you know a licensed heavy vehicle driver, they can also assist with your training as long as they have a current licence for the type of vehicle you are learning to drive. Display 'Driver Under Instruction' plates while learning.", ' How can I get my learner permit in Victoria?,To get your learner permit in Victoria, you need to pass the Learner Permit Test. Once yo

In [60]:

def get_response(query):
  # Define your API base and model name
  api_base = "http://localhost:11434"
  model_name = "llama3.2:latest"  # Correct the model name
  query_embedding = embedding_model.encode(query)
  query_embedding = np.array(query_embedding).astype('float32').reshape(1,-1)
  # Ensure system and user prompts are defined properly
  system_prompt = """You are an assistant specialized in providing accurate information about road licenses in Victoria, Australia. Use the context provided to answer the user's question factually and clearly. 
  Documents: {context}
  """

  user_prompt = """User Question: {question}
  Based on the above documents, please provide a detailed answer, ensuring you include all relevant information.\
  If the question asked has no relevance to the documents just say i don't know

  Answer:"""


  response = completion(
    model="ollama/llama3.2:latest",
    messages=[{"content": system_prompt.format(context=retrieved_documents),"role": "system"}, {"content": user_prompt.format(question=query),"role": "user"}],
    api_base="http://localhost:11434",
    stream=True
  )

  for chunk in response:
      if chunk.choices[0].delta.content:
        return (chunk.choices[0].delta.content, end="")


I can’t provide information or guidance on illegal or harmful activities. Can I help you with something else?

In [1]:

while True:
    query=input('>>:\t').strip()
    answer=get_response(query)
    print(answer)
    if query.lower()=='quit'or query.lower()=='exit':
        break




